In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder


In [2]:
traindata = np.loadtxt('/kaggle/input/limit-orderbook-data/Train_Dst_NoAuction_DecPre_CF_7.txt')
cvdata = np.loadtxt('/kaggle/input/limit-orderbook-data/Test_Dst_NoAuction_DecPre_CF_9.txt')

In [3]:
traindata.shape,cvdata.shape

In [4]:
df = pd.DataFrame(traindata[:40,:].T)
dfcv = pd.DataFrame(cvdata[:40,:].T)

In [5]:
len(dfcv)

In [6]:
# after 10 events 
sns.histplot(traindata[144,:],discrete = True)
plt.show()   

In [7]:
sns.histplot(traindata[147,:],discrete = True)
plt.show()

In [8]:
def gendata (data,k,sp):
    X = []
    Y = []
    xdat = pd.DataFrame( data[:40,:].T )
    n = int(sp*len(xdat))
    xdat = xdat[:n]
    m = len(xdat)
    epo = m - k + 1
    for i in tqdm(range(epo)):
        X.append(xdat.iloc[i:i+k,:].values)
        Y.append(data[147,i+k - 1])
    return np.array(X),np.array(Y)

In [9]:
X,Y = gendata(traindata,100,0.8)
Xcv,Ycv = gendata(cvdata,100,1)

In [10]:
X = X.reshape(X.shape[0],100, 40,1)
Xcv = Xcv.reshape(Xcv.shape[0],100,40,1)

In [11]:
X = X.astype(np.float32)

In [12]:
Xcv = Xcv.astype(np.float32)

In [13]:
ohe = OneHotEncoder()
Yn = ohe.fit_transform(Y.reshape(-1,1)).toarray()
Ycvn = ohe.fit_transform(Ycv.reshape(-1,1)).toarray()

In [14]:
Ycvn.shape

In [15]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential,Model

In [16]:
#Deep Lob Model 
In = Input(shape = (100,40,1))


# conv block 1
x = Conv2D(16,kernel_size=(1,2),strides=(1,2))(In)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = BatchNormalization()(x)

#Conv block 2 
x = Conv2D(16,kernel_size=(1,2),strides=(1,2))(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = BatchNormalization()(x)

#Conv block 3
x = Conv2D(16,kernel_size=(1,10))(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = BatchNormalization()(x)



#Inception Module

x1 = Conv2D(32,kernel_size=(1,1),padding = 'same')(x)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Conv2D(32,kernel_size=(3,1),padding = 'same')(x1)
x1 = LeakyReLU(alpha=0.05)(x1)


x2 = Conv2D(32,kernel_size=(1,1),padding = 'same')(x)
x2 = LeakyReLU(alpha=0.05)(x2)
x2 = Conv2D(32,kernel_size=(5,1),padding = 'same')(x2)
x2 = LeakyReLU(alpha=0.05)(x2)


x3 = MaxPooling2D((3,1),strides = (1,1),padding = 'same')(x)
x3 = Conv2D(32,kernel_size=(1,1))(x3)
x3 = LeakyReLU(alpha=0.05)(x3)





x = Concatenate()([x1,x2,x3])
# x = Dropout(0.2)(x)


x = Reshape((100,96))(x)


x = BatchNormalization()(x)
x = LSTM(64)(x)

x = Dense(3,activation = 'softmax')(x)
mod = Model(In,x,name= 'LOB')
mod.summary()

In [17]:
# creating a new model 


In = Input(shape = (100,40,1))


# conv block 1
x = Conv2D(16,kernel_size=(1,2),strides=(1,2))(In)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = BatchNormalization()(x)

#Conv block 2 
x = Conv2D(16,kernel_size=(1,2),strides=(1,2))(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = BatchNormalization()(x)


#Conv block 3
x = Conv2D(16,kernel_size=(1,10))(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Conv2D(16,kernel_size=(4,1),padding = 'same')(x)
x = LeakyReLU(alpha=0.05)(x)
x = BatchNormalization()(x)


#Dynamic Conv block 

x1 = Conv2D(32,kernel_size = (10,1), padding = 'same')(x)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Reshape((1,100,1,32))(x1)

x2 = Conv2D(32,kernel_size = (5,1), padding = 'same')(x)
x2 = LeakyReLU(alpha=0.05)(x2)
x2 = Reshape((1,100,1,32))(x2)

x3 = Conv2D(32,kernel_size = (3,1), padding = 'same')(x)
x3 = LeakyReLU(alpha=0.05)(x3)
x3 = Reshape((1,100,1,32))(x3)

xat = AveragePooling2D(pool_size = (10,1))(x)
xat = Flatten()(xat)
xat = Dense(64, activation = 'relu')(xat)
xat = Dense(3, activation = 'softmax' )(xat)

xres = Concatenate(axis = 1)([x1,x2,x2])
xres = Dot(axes= 1)([xat,xres])







# x = Reshape((100,32))(xres)
x = Flatten()(xres)
x = BatchNormalization()(x)
#x = LSTM(64)(x)

x = Dense(3,activation = 'softmax')(x)
mod = Model(In,x,name= 'LOB')
mod.summary()



In [18]:
from tensorflow.keras.optimizers import Adam
mod.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 1e-3), metrics = ['categorical_accuracy'])

In [19]:
hist = mod.fit(X,Yn, batch_size = 512,epochs = 20, validation_data=(Xcv,Ycvn),validation_batch_size= 64) 

In [20]:
X.shape,Yn.shape,Xcv.shape,Ycvn.shape

In [21]:
from sklearn.metrics import confusion_matrix,classification_report

In [22]:
yp = mod.predict(X)

In [23]:
print ( classification_report(np.argmax(yp,axis = 1),Y-1) ) 

In [24]:
ypval = mod.predict(Xcv)

In [25]:
print(classification_report(np.argmax(ypval,axis = 1),Ycv-1))

In [26]:
mod.save_weights('goodmod.h5')

In [27]:
del mod